<a href="https://colab.research.google.com/github/TUNIX17/LISTA/blob/main/Hotel_booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: upload a file

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle

# Load the dataset
data = pd.read_csv('transformed_dataset1.csv')

# Define feature columns and target
feature_columns = [
    'no_of_adults', 'no_of_children', 'type_of_meal_plan', 'required_car_parking_space',
    'room_type_reserved', 'lead_time', 'market_segment_type', 'repeated_guest',
    'avg_price_per_room', 'no_of_special_requests', 'total_of_days', 'No_of_week_days', 'No_of_weekend_days'
]
target_column = 'booking_status'

# Split the data into features and target
X = data[feature_columns]
y = data[target_column].apply(lambda x: 1 if x == 'Canceled' else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing for numeric and categorical columns
numeric_features = [
    'no_of_adults', 'no_of_children', 'lead_time', 'avg_price_per_room',
    'no_of_special_requests', 'total_of_days', 'No_of_week_days', 'No_of_weekend_days'
]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_features = [
    'type_of_meal_plan', 'room_type_reserved', 'market_segment_type',
    'required_car_parking_space', 'repeated_guest'
]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the entire pipeline
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
